In [2]:
# -*- coding: utf-8 -*-

import os
import re
import codecs

poliMorfPath = '../datasets/polish_dictionaries/PoliMorf-0.6.7.tab'
thesaurusPath = '../datasets/polish_dictionaries/thesaurus.txt-latest'

In [3]:
# Read all the poliMorf data
poliMorf = codecs.open(poliMorfPath,'r', encoding='UTF-8')

flexed2root = dict()
root2flexed = dict()

for line in poliMorf.readlines():
    splitted = line.split('\t')
    if (splitted[0] in flexed2root):
        temp = flexed2root[splitted[0]]
        temp.append((splitted[1], splitted[2]))
        splitted[0] = temp
    else:
        flexed2root[splitted[0]] = [(splitted[1], splitted[2])]
        
    root2flexed[splitted[1], splitted[2]] = [splitted[0]]
    
poliMorf.close()

In [4]:
# Read all the thesaurus data
thesaurus = codecs.open(thesaurusPath,'r', encoding='UTF-8')

thesaurus_dict = dict()

i = 0
for line in thesaurus.readlines():
    i = i + 1
    if i < 9:
        continue
    splitted = line.lower().split(';')
    synonyms = splitted[1:]
    
    synonyms_cleaned = []
    for item in synonyms:
        #item = re.sub('\(.*\)', '', item)
        #item = item.strip()
        if item.find('(') == -1:
            item = item.strip()
            synonyms_cleaned.append(item)
    
    thesaurus_dict[splitted[0]] = synonyms_cleaned
    i = i + 1
    
thesaurus.close()

black_list = ['do', 'na', 'nad']

for item in black_list:
    thesaurus_dict[item] = [item]

In [5]:
def find_synonyms_and_form(word):
    if not word in flexed2root:
        return []
        
    root_list = flexed2root[word]
    synonyms_forms_list = []
    for root, form in root_list:
        if root in thesaurus_dict:
            synonyms_forms_list.append((thesaurus_dict[root], form))
    return synonyms_forms_list

In [7]:

test = find_synonyms_and_form('bez')
test = [x[0] for x in test]
print(test)

[['lilak'], ['lilak'], ['lilak']]


In [8]:
import itertools
import random

def generate_sentences(sentence, max_sentences_num = 10):
    words = sentence.lower().split(' ')
    words = [[x] for x in words]
    
    for i in range(len(words)):
        synonyms = find_synonyms_and_form(words[i][0])
        
        unique_synonyms = []
        for syn_form in synonyms:
            for item in syn_form[0]:
                unique_synonyms.append(item)
        unique_synonyms = list(set(unique_synonyms))
        words[i] = words[i] + unique_synonyms
        
    
    new_sentences = []
    for i in range(max_sentences_num):
        txt = ''
        for j in range(len(words)):
            txt = txt + ' ' + random.sample(words[j],1)[0]
        txt = txt.strip()
        new_sentences.append(txt)

    
    return new_sentences

In [9]:
#print(generate_sentences('Idę do lekarza', 10))
words = generate_sentences('Idę do lekarza', 3)
print(words)

['idę do znachor', 'powodzić się, wieść się do doktor', 'idę do doktor']


In [11]:
DATASETS_DIR = '../datasets/'
ING_DIR = DATASETS_DIR+'ING-queries/'

ing_queries_file = 'queries_2017_06_29.tsv'

print('Loadig ING queries.')

import codecs
file = codecs.open(ING_DIR+ing_queries_file,'r', encoding='UTF-8')
queries= file.readlines()

print('Found %s queries.' % len(queries))

queries[4]

Loadig ING queries.
Found 248 queries.


'blablalablablba blablabla przelewu ekspresowego blablabla\tPrzelew złotówkowy\tPrzelew\tzłotówkowy\r\n'

In [10]:
filter_out = '[.!]'

output = codecs.open(ING_DIR+'test_remove.tsv','w', encoding='UTF-8')

for query in queries:
    domain = query.split('\t')[1]
    intent = query.split('\t')[2]
    something = query.split('\t')[3]
    query = query.split('\t')[0]
    query = re.sub(filter_out, '', query)
    
    augmented = generate_sentences(query, 3)
    first = query+'\t'+domain+'\t'+intent+'\t'+something.strip()+'\tN'
    #print(first.strip())
    output.write(first.strip()+'\n')
    for item in augmented:
        additional = item+'\t'+domain+'\t'+intent+'\t'+something.strip()+'\tA'
        #print(additional.strip())
        output.write(additional.strip()+'\n')
        
        
output.close()

In [15]:
print(queries[50].split('\t')[0])
generate_sentences(queries[50].split('\t')[0], 10)

Ja mam pytanie. Chciałabym przelać kwotę do koleżanki na konto walutowe.


['ja bałamucić pytanie. chciałabym przelać kwotę do koleżanki na rachunek rozliczeniowy walutowe.',
 'ja zwodzić pytanie. chciałabym przelać liczba do partnerka na rachunek rozliczeniowy walutowe.',
 'ja bałamucić pytanie. chciałabym przelać kwotę do towarzyszka na konto walutowe.',
 'ja przechodzić pytanie. chciałabym przelać kwotę do partnerka na rachunek rozliczeniowy walutowe.',
 'ja łudzić pytanie. chciałabym przelać kwotę do towarzyszka na konto walutowe.',
 'ja hipnotyzować pytanie. chciałabym przelać ilość do koleżanki na rachunek rozliczeniowy walutowe.',
 'ja bałamucić pytanie. chciałabym przelać liczba do znajoma na rachunek rozliczeniowy walutowe.',
 'ja łudzić pytanie. chciałabym przelać ilość do koleżanki na konto walutowe.',
 'ja cierpieć pytanie. chciałabym przelać wolumen do koleżanki na rachunek rozliczeniowy walutowe.',
 'ja zwodzić pytanie. chciałabym przelać liczba do koleżanki na rachunek rozliczeniowy walutowe.']